# Gradient Descent

## 1. Load data

In [1]:
import numpy as np

X_train = np.array([[1, 1], [2, 2], [2, 9], [1, 14]])
X_train.shape #m, n

y_train = np.array([3, 4, 10, 13])
y_train.shape #m, 

X_val = np.array([[1.5, 2], [2, 3], [1, 9], [1, 13]])
X_val.shape #m, n

y_val = np.array([3, 4, 9.5, 13.5])
y_val.shape #m, 

X_test = np.array([[2, 1], [1, 2], [2, 8], [2, 13]])
X_test.shape #m, n

y_test = np.array([2, 5, 9, 14])
y_test.shape #m, 

# assert X_train.shape[1] == X_test.shape[1]


(4,)

## 2. Modeling

Imagine you do:
1. imputation - cleaning
2. scaling

In [2]:
# for definitions
m_train, n_train = X_train.shape
m_val, n_val     = X_val.shape
m_test, n_test   = X_test.shape
num_epochs       = 5
theta            = np.zeros(  (n_train,   )  )
lr               = 0.001

In [3]:
#1. loop according to epoch
for i in range(num_epochs):  #0, 1, 2, 3, 4
    
    #2. predict
    yhat = X_train @ theta # (m, n) x (n, 1) = (m, 1)  #<---same shape as y 
    
    #3. gradient
    #X^T (h - y)
    grad = (X_train.T @ (yhat - y_train) ) / m_train  #(n, m) @ (m, 1) = (n, 1)
    
    #4. update
    theta = theta - lr * grad  # (n, 1) - (1) (n, 1) = (n, 1)
    
    train_loss =  (  ( yhat - y_train ) ** 2   ).sum() / m_train

    #validation loss
    #1. take the current theta, and do prediction with the validation set
    yhat_val = X_val @ theta
    #2. calculate the loss with y_val
    val_loss = (  ( yhat_val - y_val ) ** 2   ).sum() / m_val
    #3. finish
    
    print(f"Epoch: {i}  : {train_loss=:.3f}")

Epoch: 0  : train_loss=73.500
Epoch: 1  : train_loss=63.614
Epoch: 2  : train_loss=55.096
Epoch: 3  : train_loss=47.755
Epoch: 4  : train_loss=41.428
